---

# Training Multilayer perception (Geotag - Latent space mapping)

---

In [1]:
import pandas as pd
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras import layers, models
import IPython.display as ipd
from tensorflow.keras.models import load_model

## Dataset

### Load audio file path-geotag mappings

In [2]:
df = pd.read_csv('../corpus/mappings.csv')

In [3]:
df.head()

,path,lat,long
0,../corpus/audio/169884.mp3,0.501047,0.041804
1,../corpus/audio/169885.mp3,0.501052,0.041806
2,../corpus/audio/697381.mp3,0.187898,0.534158
3,../corpus/audio/187893.mp3,0.459852,0.012109
4,../corpus/audio/788102.mp3,0.236748,-0.876991


### Load audio files
- Crop audio to 44100 samples.
- Pad audio to 44100 samples.

In [4]:
audio = list()

for i, row in df.iterrows():
    y, sr = librosa.load(row['path'])

    y = np.pad(y, (0, max(0, 44100 - len(y))), mode="constant")[:44100]

    audio.append(y)

In [5]:
y = np.array(audio)
y.shape

(105, 44100)

### Get input vectors

In [6]:
X = df[['lat', 'long']].to_numpy()

## Get geotag-latent mapping

In [7]:
encoder = load_model('../models/encoder.h5')

In [8]:
latents = encoder.predict(y)

4/4 [==============================] - 1s 219ms/step


## Multilayer perceptron

In [38]:
mlp = tf.keras.Sequential()
mlp.add(layers.Input(shape=(2,)))
mlp.add(layers.Dense(64, activation='relu'))
mlp.add(layers.Dense(128, activation='relu'))
mlp.add(layers.Dense(256, activation='relu'))
mlp.add(layers.Dense(128, activation='relu'))
mlp.add(layers.Dense(64, activation='relu'))
mlp.add(layers.Dense(64, activation='tanh'))

In [39]:
mlp.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 64)                192       
                                                                 
 dense_14 (Dense)            (None, 128)               8320      
                                                                 
 dense_15 (Dense)            (None, 256)               33024     
                                                                 
 dense_16 (Dense)            (None, 128)               32896     
                                                                 
 dense_17 (Dense)            (None, 64)                8256      
                                                                 
 dense_18 (Dense)            (None, 64)                4160      
                                                                 
Total params: 86,848
Trainable params: 86,848
Non-trai

In [40]:
mlp.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mean_squared_error')

In [45]:
mlp.fit(X, latents, epochs=50)

Epoch 1/50
4/4 [==============================] - 0s 3ms/step - loss: 0.3298
Epoch 2/50
4/4 [==============================] - 0s 2ms/step - loss: 0.3297
Epoch 3/50
4/4 [==============================] - 0s 2ms/step - loss: 0.3296
Epoch 4/50
4/4 [==============================] - 0s 3ms/step - loss: 0.3302
Epoch 5/50
4/4 [==============================] - 0s 3ms/step - loss: 0.3314
Epoch 6/50
4/4 [==============================] - 0s 2ms/step - loss: 0.3303
Epoch 7/50
4/4 [==============================] - 0s 2ms/step - loss: 0.3305
Epoch 8/50
4/4 [==============================] - 0s 3ms/step - loss: 0.3297
Epoch 9/50
4/4 [==============================] - 0s 3ms/step - loss: 0.3297
Epoch 10/50
4/4 [==============================] - 0s 4ms/step - loss: 0.3291
Epoch 11/50
4/4 [==============================] - 0s 3ms/step - loss: 0.3296
Epoch 12/50
4/4 [==============================] - 0s 3ms/step - loss: 0.3303
Epoch 13/50
4/4 [==============================] - 0s 2ms/step - loss: 0.

## Test

In [49]:
decoder = load_model('../models/decoder.h5')

In [50]:
example_geotag = np.array([X[0]])

In [51]:
latent = mlp.predict(example_geotag)
audio = decoder.predict(latent)

1/1 [==============================] - 0s 51ms/step


In [52]:
ipd.display(ipd.Audio(data=audio, rate=44100))

In [54]:
mlp.save('../models/mlp.h5')